# MTurk: Approve / Reject Assignments + Post-Task Completion Processing of HITs

In [1]:
import boto3
import json
import pandas as pd
import ast
from simple_colors import *
from typing import Dict
import re
import pprint
import string

In [2]:
from mturk_processing_utils import *

In [ ]:
batch_data = [
    {
        'raw_data_path': '/Users/nehasrikanth/Documents/paraphrase-nlu/raw-data/defeasible-nli/defeasible-atomic/',
        'data_source': 'atomic',
        'creation_metadata_path': 'mturk_data/creation/atomic_dnli_annotation_examples_1.json'
    },
    {
        'raw_data_path': '/Users/nehasrikanth/Documents/paraphrase-nlu/raw-data/defeasible-nli/defeasible-atomic/',
        'data_source': 'atomic',
        'creation_metadata_path': 'mturk_data/creation/atomic_dnli_annotation_examples_2.json'
    },
    {
        'raw_data_path': '/Users/nehasrikanth/Documents/paraphrase-nlu/raw-data/defeasible-nli/defeasible-snli/',
        'data_source': 'snli',
        'creation_metadata_path': 'mturk_data/creation/snli_dnli_annotation_examples_1.json'
    },
    {
        'raw_data_path': '/Users/nehasrikanth/Documents/paraphrase-nlu/raw-data/defeasible-nli/defeasible-snli/',
        'data_source': 'snli',
        'creation_metadata_path': 'mturk_data/creation/snli_dnli_annotation_examples_2.json'
    },
    {
        'raw_data_path': '/Users/nehasrikanth/Documents/paraphrase-nlu/raw-data/defeasible-nli/defeasible-snli/',
        'data_source': 'snli',
        'creation_metadata_path': 'mturk_data/creation/snli_dnli_annotation_examples_3.json'
    },
    {
        'raw_data_path': '/Users/nehasrikanth/Documents/paraphrase-nlu/raw-data/defeasible-nli/defeasible-atomic/',
        'data_source': 'atomic',
        'creation_metadata_path': 'mturk_data/creation/atomic_dnli_annotation_examples_3.json'
    },
    
]

for batch in batch_data:
    print(batch['creation_metadata_path'])
    dataset, posted_tasks, hit_id_2_example_id = get_dataset_and_hit_id_dict(**batch)
    get_completetion_progress(hit_id_2_example_id)

In [ ]:
_ ,_ , hit_id_2_example_id_atomic_batch_1 = get_dataset_and_hit_id_dict(**batch_data[0])
dnli_atomic ,_ , hit_id_2_example_id_atomic_batch_2 = get_dataset_and_hit_id_dict(**batch_data[1])
_ ,_ , hit_id_2_example_id_atomic_batch_3 = get_dataset_and_hit_id_dict(**batch_data[5])


dnli_snli ,_ , hit_id_2_example_id_snli_batch_1 = get_dataset_and_hit_id_dict(**batch_data[2])
_ ,_ , hit_id_2_example_id_snli_batch_2 = get_dataset_and_hit_id_dict(**batch_data[3])
_ ,_ , hit_id_2_example_id_snli_batch_3 = get_dataset_and_hit_id_dict(**batch_data[4])

In [ ]:
from collections import defaultdict



In [ ]:
approved_snli_batch_1, _ = parse_batch(hit_id_2_example_id_snli_batch_1)
approved_snli_batch_2, _ = parse_batch(hit_id_2_example_id_snli_batch_2)
approved_snli_batch_3, _ = parse_batch(hit_id_2_example_id_snli_batch_3)

In [ ]:
dnli_snli.get_example_by_id(ex)

class ParaphrasedDefeasibleNLIExample:
    paraphrase_id: str # <example_id>.<example_annotator_id>.<h1_id>.<h2_id> for human, <example_id>.<system>.<identifiers> for generated
    original_example: DefeasibleNLIExample
    original_example_id: str
    update_paraphrase: str
    example_worker_id: Optional[int] = None
    worker_id: Optional[str] = None #mturk worker id or system
    premise_paraphrase: Optional[str] = None
    hypothesis_paraphrase: Optional[str] = None
    automatic_system_metadata: Optional[Dict[Any, Any]] = None # can contain system-specific metadata


In [ ]:
from dataclasses import asdict

paraphrased_dnli_examples = defaultdict(list)

for batch in [approved_snli_batch_1, approved_snli_batch_2, approved_snli_batch_3]:
    for ex, workers in batch.items():
        for i, (worker_id, worker_paraphrases) in enumerate(workers):
            for j, paraphrase in enumerate(worker_paraphrases.values()):
                paraphrased_dnli_examples[ex].append(asdict(ParaphrasedDefeasibleNLIExample(
                    paraphrase_id=f'{ex}.{i}.{j}',
                    original_example=dnli_snli.get_example_by_id(ex),
                    original_example_id=ex,
                    update_paraphrase=paraphrase,
                    example_worker_id=None,
                    worker_id=worker_id,
                    premise_paraphrase=None,
                    hypothesis_paraphrase=None,
                    automatic_system_metadata=None
                )))

In [ ]:
import json
with open('snli_approved.json', 'w') as f:
    json.dump(paraphrased_dnli_examples, f)

In [ ]:
view_assignment('3EJJQNKU9R5NKFVTHVRTXJCMA43HRZ', dnli_snli, hit_id_2_example_id_snli_batch_3)

In [ ]:
AUWM7LEMF3H9N, APHL3OC024851, AMBLMZYRPW92F

Wid: A18B2QUCJJV5KO, Aid 3V0Z7YWSIY0BYW9S7UTWBCVN6R22VZ==Wid: AUWM7LEMF3H9N, Aid = 36DSNE9QZ5YGE8Y3D3DAF1NWC94OJJ
Worker:  A3TACK9NUMFBZA AssignmentId:  34Z02EIMISDPPU2PNCP3R6ENC8FT03 == Worker:  AUWM7LEMF3H9N AssignmentId:  3EA3QWIZ4IV0WPJ5R2E9XAEJY2WITN



All 3 paraphrases are verbatim from another worker's paraphrases. This is only possible if one worker is filling out the HITs using two accounts. This is fraud, and I am rejecting the work. 

In [ ]:
response = mturk.create_worker_block(
    WorkerId='APHL3OC024851',
    Reason='Worker is using multiple accounts to fill out verbatim content.'
)